# Proyecto Final - MODELOS PREDICTIVOS
# Autor: Vásquez, William 8-889-868

#### Importamos las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from tabulate import tabulate



import warnings 
warnings.filterwarnings("ignore")


### Data ENSA 2021 - 2023

In [ ]:
data = pd.DataFrame({
    "Periodo" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 
                 31, 32, 33, 34, 35, 36],
    "Energia MWh": [
        232635168, 226840103, 242701941, 251443203, 251667868, 247422919,
        253772321, 251355408, 245272330, 249948530, 240632020, 248424779,
        248484404, 241942944, 254560420, 261669170, 262535423, 252361393,
        252123727, 255272872, 251293531, 250601276, 241522632, 254556267,
        250119557, 241106027, 262660841, 267785178, 282952947, 276303521,
        272573028, 278698818, 277187182, 276373692, 257876226, 279785978
    ]
})
# Convertimos la columna 'Periodo' en el índice para facilitar el manejo de series temporales
data.set_index('Periodo', inplace=True)

### Grafica del consumo mensual ENSA

In [ ]:
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')

        

plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Promedio móvil

In [ ]:
#Promedio Movil

data["Promedio Movil"] = data ["Energia MWh"].rolling(window=4).mean()

forecast_ma = data["Energia MWh"].rolling(window=4).mean().iloc[-1]

forecast_ma

In [ ]:
# MAD
mad_prom_movil = data['Energia MWh'].sub(data['Energia MWh'].rolling(window=4).mean().shift(1)).abs().mean()
print(f"MAD: {mad_prom_movil}")

# MAPE
mape_prom_movil = data['Energia MWh'].sub(data['Energia MWh'].rolling(window=4).mean().shift(1)).abs().div(data['Energia MWh']).mean() * 100
print(f"MAPE: {mape_prom_movil}%")


In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), data["Promedio Movil"].shift(1), marker='.', color='red', label='Promedio Movil')


plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Suavización Exponencial Simple

In [ ]:
# Ajustar el modelo de suavización exponencial simple
model_exp_Simple = SimpleExpSmoothing(data['Energia MWh'])
model_fit_exp_simple = model_exp_Simple.fit()

# Predecir los próximos 6 periodos
forecast_exp_simple = model_fit_exp_simple.forecast(steps = 2)
print(forecast_exp_simple)

In [ ]:
# Cálculo de errores
actual = data['Energia MWh'].iloc[-1]
fitted_values_expsimple = model_fit_exp_simple.fittedvalues

# MAD
mad_exp_simple = np.mean(np.abs(data['Energia MWh'] - fitted_values_expsimple))
print(f"MAD: {mad_exp_simple}")

# MAPE
mape_exp_simple = np.mean(np.abs((data['Energia MWh'] - fitted_values_expsimple) / data['Energia MWh'])) * 100
print(f"MAPE: {mape_exp_simple}%")


In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_expsimple, marker='.', color='red', label='Suavización Exponencial Simple')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Modelo Holt

In [ ]:
# Ajustar el modelo Holt
model_holt = Holt(data['Energia MWh']).fit()

# Pronóstico
forecast_holt = model_holt.forecast(steps=8)
print(forecast_holt)


In [ ]:
# Cálculo de errores
fitted_values_holt = model_holt.fittedvalues

# MAD
mad_holt = np.mean(np.abs(data['Energia MWh'] - fitted_values_holt))
print(f"MAD: {mad_holt}")

# MAPE
mape_holt = np.mean(np.abs((data['Energia MWh'] - fitted_values_holt) / data['Energia MWh'])) * 100
print(f"MAPE: {mape_holt}%")

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_holt, marker='.', color='red', label='Holt')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Modelo Holt-Winters

In [ ]:
model_HW = ExponentialSmoothing(data['Energia MWh'], seasonal='add', seasonal_periods=12).fit()
forecast_HW = model_HW.forecast(steps=8)
print(forecast_HW)

In [ ]:
# Cálculo de errores
fitted_values_HW = model_HW.fittedvalues

# MAD
mad_HW = np.mean(np.abs(data['Energia MWh'] - fitted_values_HW))
print(f"MAD: {mad_HW}")

# MAPE
mape_HW = np.mean(np.abs((data['Energia MWh'] - fitted_values_HW) / data['Energia MWh'])) * 100
print(f"MAPE: {mape_HW}%")

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_HW, marker='.', color='red', label='Holt-Winter')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Regresión Lineal

In [ ]:
X = data.index.values.reshape(-1, 1)
y = data['Energia MWh']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model_RL = LinearRegression().fit(X_train, y_train)
predictions_RL = model_RL.predict(X_test)
print(predictions_RL)

In [ ]:
# Cálculo de errores
fitted_values_RL = model_RL.predict(X)

# MAD
mad_RL = np.mean(np.abs(y - fitted_values_RL))
print(f"MAD: {mad_RL}")

# MAPE
mape_RL = np.mean(np.abs((y - fitted_values_RL) / y)) * 100
print(f"MAPE: {mape_RL}%")

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 5))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_RL, marker='.', color='red', label='Regresión Lineal')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Modelo Sarima

In [ ]:
model_SARIMAX = SARIMAX(data['Energia MWh'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)).fit()
forecast_SARIMAX = model_SARIMAX.forecast(steps=8)
print(forecast_SARIMAX)

In [ ]:
# Cálculo de errores
fitted_values_SARIMAX = model_SARIMAX.fittedvalues

# MAD
mad_SARIMAX = np.mean(np.abs(data['Energia MWh'] - fitted_values_SARIMAX))
print(f"MAD: {mad_SARIMAX}")

# MAPE
mape_SARIMAX = np.mean(np.abs((data['Energia MWh'] - fitted_values_SARIMAX) / data['Energia MWh'])) * 100
print(f"MAPE: {mape_SARIMAX}%")

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_SARIMAX, marker='.', color='red', label='SARIMAX')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

### Modelo Prophet

In [ ]:

data1 = {
    'ds': ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01',
           '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
           '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01',
           '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
           '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01',
           '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01',
           '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01',
           '2023-12-01'],
    'y': [232635168, 226840103, 242701941, 251443203, 251667868, 247422919, 253772321,
          251355408, 245272330, 249948530, 240632020, 248424779, 248484404, 241942944,
          254560420, 261669170, 262535423, 252361393, 252123727, 255272872, 251293531,
          250601276, 241522632, 254556267, 250119557, 241106027, 262660841, 267785178,
          282952947, 276303521, 272573028, 278698818, 277187182, 276373692, 257876226,
          279785978]
}

df = pd.DataFrame(data1)

# Convertir la columna de fechas a tipo datetime
df['ds'] = pd.to_datetime(df['ds'])

# Inicializar y ajustar el modelo
model = Prophet()
model.fit(df)

# Crear un DataFrame para futuras predicciones (ajusta periods según tus necesidades)
future = model.make_future_dataframe(periods=12, freq='M')

# Usar el modelo para hacer predicciones
forecast_prophet = model.predict(future)

# Imprimir o graficar las predicciones según sea necesario
print(forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])


In [ ]:
# Cálculo de errores
fitted_values_prophet = model.predict(df)['yhat']

# MAD
mad_prophet = np.mean(np.abs(df['y'] - fitted_values_prophet))
print(f"MAD: {mad_prophet}")

# MAPE
mape_prophet = np.mean(np.abs((df['y'] - fitted_values_prophet) / df['y'])) * 100
print(f"MAPE: {mape_prophet}%")

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data1['y'], marker='.', color='blue', label='Consumo Histórico')
plt.plot(list(range(1,37)), fitted_values_prophet, marker='.', color='red', label='Prophet')

# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('ENSA 2021 - 2023')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

In [ ]:
#Graficamos
# Crear la figura y los ejes
plt.figure(figsize=(10, 6))

# Graficar los datos históricos
plt.plot(list(range(1,37)), data['Energia MWh'], marker='.', color='blue', label='Consumo Histórico')


# Graficar la predicción de promedio movil
plt.scatter(37, 272805770, color='red', s=50, marker='H', label='Predicción Promedio Movil')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1], 272805770], color='red')  # Línea para conectar el último punto histórico con la predicción

#Graficar la prediccion de suavizacion exponencial simple 
plt.scatter(37, 2.745158e+08, color='aqua', s=50, marker='H', label='Predicción Suavización Exponencial Simple')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1], 2.745158e+08], color='aqua')  # Línea para conectar el último punto histórico con la predicción

periodos_prediccion = list(range(37, 45))

#Graficar las predicciones Holt
plt.plot(periodos_prediccion, forecast_holt.values, marker='o', color='green', label='Predicción Holt')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1], 2.771269e+08 ], color='green')

#Graficar las predicciones Holt-Winter
plt.plot(periodos_prediccion, forecast_HW.values, marker='o', color='deeppink', label='Predicción Holt-Winter')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1], 2.736294e+08 ], color='deeppink')

#Graficar las predicciones Sarimax
plt.plot(periodos_prediccion, forecast_SARIMAX.values, marker='o', color='chartreuse', label='Predicción Sarimax')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1],  2.717389e+08 ], color='chartreuse')

#Graficar las predicciones Prophet
plt.plot(periodos_prediccion, forecast_prophet["yhat"].loc[36:43] , marker='o', color='darkmagenta', label='Predicción Prophet')
plt.plot([36, 37], [data['Energia MWh'].iloc[-1],  2.771098e+08 ], color='darkmagenta')



# Personalizar la gráfica
plt.xlabel('Periodo')
plt.ylabel('Energía MWh')
plt.title('Consumo de Energía en Panamá (ENSA)')
plt.legend()
plt.grid(True)

# Mostrar la gráfica
plt.show()

In [ ]:
# Crear un DataFrame con las métricas
dict_metricas = {
    'Modelo': ['Promedio Movil', 'Suavización Exponencial', 'Holt', 'Holt-Winter', 'Sarimax', 'Regresión', 'Prophet'],
    'MAD': [mad_prom_movil, mad_exp_simple, mad_holt, mad_HW, mad_SARIMAX, mad_RL, mad_prophet],
    'MAPE': [mape_prom_movil, mape_exp_simple, mape_holt, mape_HW, mape_SARIMAX, mape_RL, mape_prophet],
    'STD': [mad_prom_movil*1.25, mad_exp_simple*1.25, mad_holt*1.25, mad_HW*1.25, mad_SARIMAX*1.25, mad_RL*1.25, mad_prophet*1.25]
}

df_metricas = pd.DataFrame(dict_metricas)

# Convertir el DataFrame a formato tabulado
table = tabulate(df_metricas, headers='keys', tablefmt='fancy_grid')

# Mostrar la tabla formateada
print(table)